# Changes in HbA1c Levels During COVID-19 

This notebook describes trends and deviations in HbA1c test results during the COVID-19 pandemic at large and by specific demographic groups. The following measures are reported: 
1. Absolute number of abnormal HbA1c tests
2. Population mean of test results
3. Deviations from the population mean

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

from IPython.core.display import HTML as Center
from IPython.display import HTML

In [30]:
# Import function
def import_df(f_path):
    df = pd.read_csv("../output/{}.csv".format(f_path))
    # Create percent of population
    df['pct'] = df['value']*100
    df['year'] = df['date'].apply(lambda x: x[:4])
    return df

In [31]:
# Dataframes
df_pop = import_df('measure_hba1c_abnormal')
df_dm = import_df('measure_hba1c_abnormal_by_diabetes_type')
df_sex = import_df('measure_hba1c_abnormal_by_sex')
df_age = import_df('measure_hba1c_abnormal_by_age')
df_region = import_df('measure_hba1c_abnormal_by_region')
df_imd = import_df('measure_hba1c_abnormal_by_imd')

## Abnormal Results (Means by Year)

In [40]:
def mean_abnormal(df,measure):
    display(pd.DataFrame(
        df.groupby(['year', measure])['hba1c_abnormal'].mean()
    ))

In [42]:
# Diabetes type
mean_abnormal(df_dm, 'diabetes_type')

hba1c_abnormal
year diabetes_type                
2019 NO_DM              444.166667
     T1DM                19.750000
     T2DM               116.250000
     UNKNOWN_DM          12.083333
2020 NO_DM              450.750000
     T1DM                18.416667
     T2DM               116.333333
     UNKNOWN_DM          11.000000
2021 NO_DM              439.833333
     T1DM                16.000000
     T2DM               125.500000
     UNKNOWN_DM          12.500000

In [43]:
# Sex
mean_abnormal(df_sex, 'sex')

hba1c_abnormal
year sex                
2019 F        297.916667
     M        294.333333
2020 F        301.166667
     M        295.333333
2021 F        298.833333
     M        295.000000

In [44]:
# Age
mean_abnormal(df_age, 'age_group')

hba1c_abnormal
year age_group                
2019 0-19           110.000000
     20-29           60.916667
     30-39           63.416667
     40-49           93.333333
     50-59           59.583333
     60-69           57.083333
     70-79           57.250000
     80+             78.916667
     missing         11.750000
2020 0-19           120.250000
     20-29           56.666667
     30-39           61.416667
     40-49           89.750000
     50-59           62.000000
     60-69           58.000000
     70-79           61.333333
     80+             74.750000
     missing         12.333333
2021 0-19           122.500000
     20-29           55.833333
     30-39           57.666667
     40-49           92.833333
     50-59           60.833333
     60-69           53.833333
     70-79           62.000000
     80+             77.166667
     missing         11.166667

In [45]:
# Age
mean_abnormal(df_region, 'region')

hba1c_abnormal
year region                                  
2019 East Midlands                  44.833333
     East of England                49.666667
     London                         93.750000
     North East                     48.583333
     North West                     45.083333
     South East                     95.416667
     West Midlands                  49.416667
     Yorkshire and the Humber       46.166667
2020 East Midlands                  44.333333
     East of England                46.750000
     London                        100.166667
     North East                     50.750000
     North West                     48.083333
     South East                     94.666667
     West Midlands                  47.833333
     Yorkshire and the Humber       46.500000
2021 East Midlands                  46.666667
     East of England                49.833333
     London                         90.000000
     North East                     49.833333
     North West                     42.500000
     South East                    100.833333
     West Midlands                  52.666667
     Yorkshire and the Humber       44.000000

In [46]:
# IMD 
mean_abnormal(df_imd, 'imd')

hba1c_abnormal
year imd                
2019 0          5.916667
     1        115.833333
     2        122.750000
     3        119.833333
     4        116.916667
     5        111.000000
2020 0          5.166667
     1        119.666667
     2        127.333333
     3        120.333333
     4        115.666667
     5        108.333333
2021 0          5.333333
     1        115.666667
     2        125.000000
     3        115.166667
     4        121.000000
     5        111.666667